In [1]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
# Vlez: I'm a beginner, too.
# Izlez: Yo también soy principiante.

# <START> Yo también soy principiante. <END>

# Vlezna rechenica       Prethodni zborovi                     Sleden zbor
# I'm a beginner, too.   <START>                               Yo
# I'm a beginner, too.   <START> Yo                            también
# I'm a beginner, too.   <START> Yo también                    soy
# I'm a beginner, too.   <START> Yo también soy                principiante
# I'm a beginner, too.   <START> Yo también soy principiante.  <END>



In [3]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
from seq2seq import create_train_data, create_model, decode, convert
from mt_utils import tokenize, append_start_end_token, create_vocabulary, map_to_index

In [4]:
data = pd.read_csv('en_es_corpus.txt', sep='\t', header=None)
data = data[[0, 1]]
data.columns = ['EN', 'ES']
data = data[:1000]
data

,EN,ES
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
...,...,...
995,Stay away.,Fuera.
996,Stay away.,Aléjate.
997,Stay calm.,Mantén la calma.
998,Stay calm.,Quedate tranquilo.


In [5]:
tokenize(data)

In [6]:
data

,EN,ES,EN_tokens,ES_tokens
0,Go.,Ve.,"[go, .]","[ve, .]"
1,Go.,Vete.,"[go, .]","[vete, .]"
2,Go.,Vaya.,"[go, .]","[vaya, .]"
3,Go.,Váyase.,"[go, .]","[váyase, .]"
4,Hi.,Hola.,"[hi, .]","[hola, .]"
...,...,...,...,...
995,Stay away.,Fuera.,"[stay, away, .]","[fuera, .]"
996,Stay away.,Aléjate.,"[stay, away, .]","[aléjate, .]"
997,Stay calm.,Mantén la calma.,"[stay, calm, .]","[mantén, la, calma, .]"
998,Stay calm.,Quedate tranquilo.,"[stay, calm, .]","[quedate, tranquilo, .]"


In [7]:
append_start_end_token(data)

In [8]:
data

,EN,ES,EN_tokens,ES_tokens
0,Go.,Ve.,"[<START>, go, ., <END>]","[<START>, ve, ., <END>]"
1,Go.,Vete.,"[<START>, go, ., <END>]","[<START>, vete, ., <END>]"
2,Go.,Vaya.,"[<START>, go, ., <END>]","[<START>, vaya, ., <END>]"
3,Go.,Váyase.,"[<START>, go, ., <END>]","[<START>, váyase, ., <END>]"
4,Hi.,Hola.,"[<START>, hi, ., <END>]","[<START>, hola, ., <END>]"
...,...,...,...,...
995,Stay away.,Fuera.,"[<START>, stay, away, ., <END>]","[<START>, fuera, ., <END>]"
996,Stay away.,Aléjate.,"[<START>, stay, away, ., <END>]","[<START>, aléjate, ., <END>]"
997,Stay calm.,Mantén la calma.,"[<START>, stay, calm, ., <END>]","[<START>, mantén, la, calma, ., <END>]"
998,Stay calm.,Quedate tranquilo.,"[<START>, stay, calm, ., <END>]","[<START>, quedate, tranquilo, ., <END>]"


In [9]:
words_en, word_to_id_en, id_to_word_en = create_vocabulary(data['EN_tokens'].values)

In [10]:
words_en

[np.str_('awful'),
 np.str_('live'),
 np.str_('write'),
 np.str_('new'),
 np.str_('him'),
 np.str_('refuse'),
 np.str_('work'),
 np.str_('safe'),
 np.str_('there'),
 np.str_('me'),
 np.str_('marry'),
 np.str_('pay'),
 np.str_('oh'),
 np.str_('give'),
 np.str_('broke'),
 np.str_('dogs'),
 np.str_('loosen'),
 np.str_('sit'),
 np.str_('answer'),
 np.str_('hope'),
 np.str_('woke'),
 np.str_('warn'),
 np.str_('thanks'),
 np.str_('go'),
 np.str_('use'),
 np.str_('late'),
 np.str_('ask'),
 np.str_('snowed'),
 np.str_('skip'),
 np.str_('full'),
 np.str_('love'),
 np.str_('done'),
 np.str_('came'),
 np.str_('above'),
 np.str_('shy'),
 np.str_('be'),
 np.str_('find'),
 np.str_('goodbye'),
 np.str_('hurt'),
 np.str_('fit'),
 np.str_('easy'),
 np.str_('back'),
 np.str_('hi'),
 np.str_('still'),
 np.str_('agreed'),
 np.str_('soon'),
 np.str_('see'),
 np.str_('changed'),
 np.str_('angry'),
 np.str_('move'),
 np.str_('calm'),
 np.str_('cried'),
 np.str_('really'),
 np.str_('cringed'),
 np.str_('old')

In [11]:
len(words_en)

378

In [12]:
word_to_id_en

{np.str_('awful'): 0,
 np.str_('live'): 1,
 np.str_('write'): 2,
 np.str_('new'): 3,
 np.str_('him'): 4,
 np.str_('refuse'): 5,
 np.str_('work'): 6,
 np.str_('safe'): 7,
 np.str_('there'): 8,
 np.str_('me'): 9,
 np.str_('marry'): 10,
 np.str_('pay'): 11,
 np.str_('oh'): 12,
 np.str_('give'): 13,
 np.str_('broke'): 14,
 np.str_('dogs'): 15,
 np.str_('loosen'): 16,
 np.str_('sit'): 17,
 np.str_('answer'): 18,
 np.str_('hope'): 19,
 np.str_('woke'): 20,
 np.str_('warn'): 21,
 np.str_('thanks'): 22,
 np.str_('go'): 23,
 np.str_('use'): 24,
 np.str_('late'): 25,
 np.str_('ask'): 26,
 np.str_('snowed'): 27,
 np.str_('skip'): 28,
 np.str_('full'): 29,
 np.str_('love'): 30,
 np.str_('done'): 31,
 np.str_('came'): 32,
 np.str_('above'): 33,
 np.str_('shy'): 34,
 np.str_('be'): 35,
 np.str_('find'): 36,
 np.str_('goodbye'): 37,
 np.str_('hurt'): 38,
 np.str_('fit'): 39,
 np.str_('easy'): 40,
 np.str_('back'): 41,
 np.str_('hi'): 42,
 np.str_('still'): 43,
 np.str_('agreed'): 44,
 np.str_('soon')

In [13]:
id_to_word_en

{0: np.str_('awful'),
 1: np.str_('live'),
 2: np.str_('write'),
 3: np.str_('new'),
 4: np.str_('him'),
 5: np.str_('refuse'),
 6: np.str_('work'),
 7: np.str_('safe'),
 8: np.str_('there'),
 9: np.str_('me'),
 10: np.str_('marry'),
 11: np.str_('pay'),
 12: np.str_('oh'),
 13: np.str_('give'),
 14: np.str_('broke'),
 15: np.str_('dogs'),
 16: np.str_('loosen'),
 17: np.str_('sit'),
 18: np.str_('answer'),
 19: np.str_('hope'),
 20: np.str_('woke'),
 21: np.str_('warn'),
 22: np.str_('thanks'),
 23: np.str_('go'),
 24: np.str_('use'),
 25: np.str_('late'),
 26: np.str_('ask'),
 27: np.str_('snowed'),
 28: np.str_('skip'),
 29: np.str_('full'),
 30: np.str_('love'),
 31: np.str_('done'),
 32: np.str_('came'),
 33: np.str_('above'),
 34: np.str_('shy'),
 35: np.str_('be'),
 36: np.str_('find'),
 37: np.str_('goodbye'),
 38: np.str_('hurt'),
 39: np.str_('fit'),
 40: np.str_('easy'),
 41: np.str_('back'),
 42: np.str_('hi'),
 43: np.str_('still'),
 44: np.str_('agreed'),
 45: np.str_('so

In [14]:
words_es, word_to_id_es, id_to_word_es = create_vocabulary(data['ES_tokens'].values)

In [15]:
words_es

[np.str_('despotriqué'),
 np.str_('ataque'),
 np.str_('lee'),
 np.str_('cerrojo'),
 np.str_('buenas'),
 np.str_('hazlo'),
 np.str_('vemos'),
 np.str_('besen'),
 np.str_('seguidme'),
 np.str_('grave'),
 np.str_('desapercibido'),
 np.str_('esquiar'),
 np.str_('gordo'),
 np.str_('cómprela'),
 np.str_('avísale'),
 np.str_('muriendo'),
 np.str_('os'),
 np.str_('firmen'),
 np.str_('me'),
 np.str_('márchate'),
 np.str_('¡capturadlo'),
 np.str_('abrigados'),
 np.str_('blanca'),
 np.str_('miren'),
 np.str_('tentá'),
 np.str_('jesús'),
 np.str_('ahora'),
 np.str_('tengo'),
 np.str_('perdí'),
 np.str_('mío'),
 np.str_('sea'),
 np.str_('agradable'),
 np.str_('¡no'),
 np.str_('pasa'),
 np.str_('alcanzaron'),
 np.str_('cuidado'),
 np.str_('tentad'),
 np.str_('rico'),
 np.str_('¡tranquilo'),
 np.str_('viviré'),
 np.str_('¡cogedlo'),
 np.str_('probó'),
 np.str_('voy'),
 np.str_('trabajo'),
 np.str_('¡cerrá'),
 np.str_('correr'),
 np.str_('caí'),
 np.str_('favor'),
 np.str_('cantaré'),
 np.str_('moment

In [16]:
len(words_es)

897

In [17]:
word_to_id_es

{np.str_('despotriqué'): 0,
 np.str_('ataque'): 1,
 np.str_('lee'): 2,
 np.str_('cerrojo'): 3,
 np.str_('buenas'): 4,
 np.str_('hazlo'): 5,
 np.str_('vemos'): 6,
 np.str_('besen'): 7,
 np.str_('seguidme'): 8,
 np.str_('grave'): 9,
 np.str_('desapercibido'): 10,
 np.str_('esquiar'): 11,
 np.str_('gordo'): 12,
 np.str_('cómprela'): 13,
 np.str_('avísale'): 14,
 np.str_('muriendo'): 15,
 np.str_('os'): 16,
 np.str_('firmen'): 17,
 np.str_('me'): 18,
 np.str_('márchate'): 19,
 np.str_('¡capturadlo'): 20,
 np.str_('abrigados'): 21,
 np.str_('blanca'): 22,
 np.str_('miren'): 23,
 np.str_('tentá'): 24,
 np.str_('jesús'): 25,
 np.str_('ahora'): 26,
 np.str_('tengo'): 27,
 np.str_('perdí'): 28,
 np.str_('mío'): 29,
 np.str_('sea'): 30,
 np.str_('agradable'): 31,
 np.str_('¡no'): 32,
 np.str_('pasa'): 33,
 np.str_('alcanzaron'): 34,
 np.str_('cuidado'): 35,
 np.str_('tentad'): 36,
 np.str_('rico'): 37,
 np.str_('¡tranquilo'): 38,
 np.str_('viviré'): 39,
 np.str_('¡cogedlo'): 40,
 np.str_('probó'

In [18]:
id_to_word_es

{0: np.str_('despotriqué'),
 1: np.str_('ataque'),
 2: np.str_('lee'),
 3: np.str_('cerrojo'),
 4: np.str_('buenas'),
 5: np.str_('hazlo'),
 6: np.str_('vemos'),
 7: np.str_('besen'),
 8: np.str_('seguidme'),
 9: np.str_('grave'),
 10: np.str_('desapercibido'),
 11: np.str_('esquiar'),
 12: np.str_('gordo'),
 13: np.str_('cómprela'),
 14: np.str_('avísale'),
 15: np.str_('muriendo'),
 16: np.str_('os'),
 17: np.str_('firmen'),
 18: np.str_('me'),
 19: np.str_('márchate'),
 20: np.str_('¡capturadlo'),
 21: np.str_('abrigados'),
 22: np.str_('blanca'),
 23: np.str_('miren'),
 24: np.str_('tentá'),
 25: np.str_('jesús'),
 26: np.str_('ahora'),
 27: np.str_('tengo'),
 28: np.str_('perdí'),
 29: np.str_('mío'),
 30: np.str_('sea'),
 31: np.str_('agradable'),
 32: np.str_('¡no'),
 33: np.str_('pasa'),
 34: np.str_('alcanzaron'),
 35: np.str_('cuidado'),
 36: np.str_('tentad'),
 37: np.str_('rico'),
 38: np.str_('¡tranquilo'),
 39: np.str_('viviré'),
 40: np.str_('¡cogedlo'),
 41: np.str_('pr

In [19]:
map_to_index(data, word_to_id_en, word_to_id_es)

In [20]:
data

,EN,ES,EN_tokens,ES_tokens,EN_index,ES_index
0,Go.,Ve.,"[<START>, go, ., <END>]","[<START>, ve, ., <END>]","[168, 23, 62, 274]","[431, 894, 137, 656]"
1,Go.,Vete.,"[<START>, go, ., <END>]","[<START>, vete, ., <END>]","[168, 23, 62, 274]","[431, 183, 137, 656]"
2,Go.,Vaya.,"[<START>, go, ., <END>]","[<START>, vaya, ., <END>]","[168, 23, 62, 274]","[431, 863, 137, 656]"
3,Go.,Váyase.,"[<START>, go, ., <END>]","[<START>, váyase, ., <END>]","[168, 23, 62, 274]","[431, 177, 137, 656]"
4,Hi.,Hola.,"[<START>, hi, ., <END>]","[<START>, hola, ., <END>]","[168, 42, 62, 274]","[431, 390, 137, 656]"
...,...,...,...,...,...,...
995,Stay away.,Fuera.,"[<START>, stay, away, ., <END>]","[<START>, fuera, ., <END>]","[168, 58, 223, 62, 274]","[431, 562, 137, 656]"
996,Stay away.,Aléjate.,"[<START>, stay, away, ., <END>]","[<START>, aléjate, ., <END>]","[168, 58, 223, 62, 274]","[431, 692, 137, 656]"
997,Stay calm.,Mantén la calma.,"[<START>, stay, calm, ., <END>]","[<START>, mantén, la, calma, ., <END>]","[168, 58, 50, 62, 274]","[431, 604, 813, 425, 137, 656]"
998,Stay calm.,Quedate tranquilo.,"[<START>, stay, calm, ., <END>]","[<START>, quedate, tranquilo, ., <END>]","[168, 58, 50, 62, 274]","[431, 396, 696, 137, 656]"


In [21]:
input_en, input_prev_es, output_es = create_train_data(data['EN_index'].values, data['ES_index'].values)

In [22]:
data['EN_index'].values[0]

[168, 23, 62, 274]

In [23]:
input_en[0]

[168, 23, 62, 274]

In [24]:
input_prev_es[0], output_es[0]

([431], 894)

In [25]:
input_prev_es[1], output_es[1]

([431, 894], 137)

In [26]:
input_prev_es[2], output_es[2]

([431, 894, 137], 656)

In [27]:
len(input_en)

3935

In [28]:
padded_input_en = pad_sequences(input_en, maxlen=10)
padded_input_en

array([[  0,   0,   0, ...,  23,  62, 274],
       [  0,   0,   0, ...,  23,  62, 274],
       [  0,   0,   0, ...,  23,  62, 274],
       ...,
       [  0,   0,   0, ...,  50,  62, 274],
       [  0,   0,   0, ...,  50,  62, 274],
       [  0,   0,   0, ...,  50,  62, 274]], dtype=int32)

In [29]:
padded_input_prev_es = pad_sequences(input_prev_es, maxlen=10)
padded_input_prev_es

array([[  0,   0,   0, ...,   0,   0, 431],
       [  0,   0,   0, ...,   0, 431, 894],
       [  0,   0,   0, ..., 431, 894, 137],
       ...,
       [  0,   0,   0, ..., 431, 325, 813],
       [  0,   0,   0, ..., 325, 813, 425],
       [  0,   0,   0, ..., 813, 425, 137]], dtype=int32)

In [30]:
label_binarizer = LabelBinarizer()

In [31]:
word_to_id_es.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [32]:
label_binarizer.fit_transform(list(word_to_id_es.values()))

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [33]:
output_es_one_hot = label_binarizer.transform(output_es)
output_es_one_hot

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [34]:
model = create_model(10, len(words_en), len(words_es), 50)

In [35]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 10, 50)    │     18,900 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 10, 50)    │     44,850 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 128),     │     91,648 │ embedding[0][0]   │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 128),     │     91,648 │ embedding_1[0][0… │
│                     │ (None, 128),      │            │ lstm[0][1],       │
│                     │ (None, 128)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 897)       │    115,713 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 362,759 (1.38 MB)

 Trainable params: 317,909 (1.21 MB)

 Non-trainable params: 44,850 (175.20 KB)

In [36]:
model.fit([padded_input_en, padded_input_prev_es],
           output_es_one_hot,
           epochs=5, batch_size=16)

Epoch 1/5
246/246 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 4.5664
Epoch 2/5
246/246 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 3.5537
Epoch 3/5
246/246 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 3.2716
Epoch 4/5
246/246 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 3.1825
Epoch 5/5
246/246 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 2.9457


In [37]:
prediction = decode(model, padded_input_en[0], word_to_id_es, 10)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


[431,
 np.int64(559),
 np.int64(137),
 np.int64(656),
 np.int64(137),
 np.int64(656),
 np.int64(137),
 np.int64(656),
 np.int64(137),
 np.int64(656),
 np.int64(137)]

In [38]:
convert([padded_input_en[0]], id_to_word_en)

['awful awful awful awful awful awful <START> go . <END>']

In [39]:
convert([prediction], id_to_word_es)

['<START> estoy . <END> . <END> . <END> . <END> .']